In [ ]:
from jinja2 import Environment, FileSystemLoader
import pandas as pd
import random
from datetime import datetime
import string
from hashlib import sha1

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IndicWiki-OlympicGames/Athletes-Data/Olympicgames_Final_Dataset.csv')
print(df.head())

   Unnamed: 0  ID                                Name  \
0           0   2                            A Lamusi   
1           1  21           Ragnhild Margrethe Aamodt   
2           2  22                       Andreea Aanei   
3           3  23                         Fritz Aanes   
4           4  34  Jamale (Djamel-) Aarrass (Ahrass-)   

                             te_Name Gender  Gender.1  Age  Height  Weight  \
0                            ఏ లముసి      M  పురుషుడు   23   170.0    60.0   
1    రాగ్న్‌హిల్డ్ మార్గరెతే అమోడ్ట్      F    స్త్రీ   27   163.0     NaN   
2                       ఆండ్రియా అనీ      F    స్త్రీ   22   170.0   125.0   
3                     ఫ్రిట్జ్ అనెస్      M  పురుషుడు   26   187.0    89.0   
4  జమాలే  (జామెల్-) ఆరాస్ (అహ్రాస్-)      M  పురుషుడు   30   187.0    76.0   

   Country  ...  Club te_Club Coach te_Coach Education te_Education Residence  \
0    China  ...  None    None  None     None      None         None      None   
1   Norway  ...  None    N

In [ ]:
print(df.columns)

Index(['Unnamed: 0', 'ID', 'Name', 'te_Name', 'Gender', 'Gender.1', 'Age',
       'Height', 'Weight', 'Country', 'te_Country', 'NOC', 'Games', 'te_Games',
       'Years', 'Seasons', 'te_Seasons', 'Cities', 'te_Cities', 'Sports',
       'te_Sports', 'Events', 'te_Events', 'Medals', 'te_Medals', 'Wiki_Links',
       'Label', 'te_Label', 'Date_Of_Birth', 'Birth Place', 'te_Birth Place',
       'StateOfOrigin', 'te_StateOfOrigin', 'Thumbnail', 'Wiki_Image_Link',
       'Club', 'te_Club', 'Coach', 'te_Coach', 'Education', 'te_Education',
       'Residence', 'te_Residence', 'Caption', 'te_Caption'],
      dtype='object')


In [ ]:
def getData(dF, ind):
  games = dF['te_Games'][ind].replace('[', '').replace(']', '').replace("'", '').split(', ')
  cities = dF['te_Cities'][ind].replace('[', '').replace(']', '').replace("'", '').split(', ')
  sports = dF['te_Sports'][ind].replace('[', '').replace(']', '').replace("'", '').split(', ')
  medals = dF['te_Medals'][ind].replace('[', '').replace(']', '').replace("'", '').split(', ')
  events = dF['te_Events'][ind].replace('[', '').replace(']', '').split('", "')
  events = [x.replace('"', '') for x in events]
  # events = ["అథ్లెటిక్స్ పురుషుల 200 మీటర్లు", "అథ్లెటిక్స్ పురుషుల 100 మీటర్లు", "అథ్లెటిక్స్ పురుషుల 200 మీటర్లు", "అథ్లెటిక్స్ పురుషుల 4 x 100 మీటర్ల రిలే", "అథ్లెటిక్స్ పురుషుల 100 మీటర్లు", "అథ్లెటిక్స్ 20 మెట్ల x 100 మీటర్ల రిలే", "అథ్లెటిక్స్ పురుషుల 4 x 100 మీటర్ల రిలే", "అథ్లెటిక్స్ పురుషుల 100 మీటర్లు", "అథ్లెటిక్స్ పురుషుల 200 మీటర్లు", "అథ్లెటిక్స్ పురుషుల 4 x 100 మీటర్ల రిలే"]
  categories = [
                '[[వర్గం:తెవికీ ఒలింపిక్ అథ్లెట్]]\n',
                '[[వర్గం:' + dF['te_Country'][ind] +' అథ్లెట్]]\n',
                '[[వర్గం:' + sports[0] +' అథ్లెట్]]\n'
  ]
  image = dF['Thumbnail'][ind]
  if image == 'None':
    image = ''
  else:
    image = dF['Thumbnail'][ind].split('/')[-1]
    if '?' in image:
      image = image[:image.index('?')]
  caption = dF['te_Caption'][ind]
  if caption == 'None':
    caption = ''
  return {
        'name': dF['te_Name'][ind],
        'image': image,
        'caption': caption,
        'Gender': dF['Gender'][ind],
        'dob': dF['Date_Of_Birth'][ind],
        'pob': dF['te_Birth Place'][ind],
        'height': str(dF['Height'][ind]),
        'weight': str(dF['Weight'][ind]),
        'country': dF['te_Country'][ind],
        'NOC': dF['NOC'][ind],
        'sport': sports[0],
        'games': games,
        'cities': cities,
        'sports': sports,
        'events': events,
        'medals': medals,
        'stateOfOrigin': dF['te_StateOfOrigin'][ind],
        'residence': dF['te_Residence'][ind],
        'education': dF['te_Education'][ind],
        'coach': dF['te_Coach'][ind],
        'club': dF['te_Club'][ind],
        'nationality': dF['te_Country'][ind],
        'categories': categories,
        'dob_ref': "{{Cite web|url=https://dbpedia.org/page/" + dF['Wiki_Links'][ind].split('/')[-1] + "|title= Dbpedia Page of " + dF['Name'][ind] + "|language=en}}"
    }

In [ ]:
def getProfessionalData(data):
  if data['Gender'] == 'M':
    v1 = 'వెనుతిరిగాడు'
    v2 = 'గెలుచుకున్నాడు'
  else:
    v1 = 'వెనుతిరిగింది'
    v2 = 'గెలుచుకుంది'
  # print(len(data['games']), len(data['cities']), len(data['sports']), len(data['events']), len(data['medals']))
  professionalLifeData = []
  i = 0
  while i < len(data['games']):
    indexs = []
    for j in range(i, len(data['games'])):
      if data['games'][j] == data['games'][i]:
        indexs.append(j)
    if indexs[-1] == 0:
      i += 1
    else:
      i = indexs[-1] + 1
    lst = [
      data['name'] + ' ' + data['games'][indexs[0]].split(' ')[0] + ' '  + "[https://te.wikipedia.org/wiki" + data['cities'][indexs[0]].replace(' ',  '_') + " " + data['cities'][indexs[0]] + "]" + " నగరంలో నిర్వహించబడిన " + data['games'][indexs[0]].split(' ')[1] + ' ఒలింపిక్స్లో' +  ' ' + data['sports'][indexs[0]] + " డిసిప్లిన్ లో ",
      "[https://te.wikipedia.org/wiki" + data['cities'][indexs[0]].replace(' ',  '_') + " " + data['cities'][indexs[0]] + "]" + " నగరంలో నిర్వహించబడిన " + data['games'][indexs[0]] + " ఒలింపిక్స్లో " +  data['name'] + ' ' + data['sports'][indexs[0]] + " డిసిప్లిన్ లో "
      ]
    info = lst[random.choice([x for x in range(100)]) % 2]
    for m in indexs:
      if indexs[0] == m:
        info += data['events'][m]
      else:
        info += ', ' + data['events'][m]
    if len(indexs) == 1:
      info += " ఈవెంట్లో పాల్గొనగా "
    else:
      info += " ఈవెంట్లలో పాల్గొనగా "
    checkIfMedal = 1
    for m in indexs:
      if(data['medals'][m] != 'nan'):
        checkIfMedal = 0
        info += ', ' + data['events'][m] + " లో " + data['medals'][m] + " [https://te.wikipedia.org/wiki/పతకం పతకం]"
    if checkIfMedal:
      info += '[https://te.wikipedia.org/wiki/పతకం పతకం] గెల్వకుండానే ' + v1 + '.'
    else:
      info += ' ' + v2 + '.'
    if data['games'][indexs[0]].split(' ')[0] != '2012' and data['games'][indexs[0]].split(' ')[0] != '2016':
      professionalLifeData.append("=== [https://te.wikipedia.org/wiki/" + data['games'][indexs[0]].split(' ')[0] + "_వేసవి_ఒలింపిక్_క్రీడలు " + data['games'][indexs[0]].split(' ')[0] + " ఒలింపిక్స్] " + "===")
    else:
      professionalLifeData.append("=== " + data['games'][indexs[0]].split(' ')[0] + " ఒలింపిక్స్ ===")
    professionalLifeData.append(info)
  data['professionalData'] = professionalLifeData
  return data

In [ ]:
def getMedalsData(data):
  if len(data['medals']) != data['medals'].count('nan'):
    medalsData = ["| medaltemplates = "]
    for i in range(len(data['medals'])):
      if data['medals'][i] == 'గోల్డ్':
        medalsData.append("{{Medal" + "Gold | " + data['games'][i].split(' ')[0] + " " + data['cities'][i] + " | " + data['events'][i] + "}}")
      elif data['medals'][i] == 'సిల్వర్':
        medalsData.append("{{Medal" + "Silver | " + data['games'][i].split(' ')[0] + " " + data['cities'][i] + " | " + data['events'][i] + "}}")
      elif data['medals'][i] == 'బ్రోన్జ్':
        medalsData.append("{{Medal" + "Bronze | " + data['games'][i].split(' ')[0] + " " + data['cities'][i] + " | " + data['events'][i] + "}}")
    medalsData.append("}}")
    medalsData = '\n'.join(medalsData)
    data['medalsData'] = medalsData
  return data

In [ ]:
tewiki = '''<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="te">
	<siteinfo>
		<sitename>tewiki</sitename>
		<dbname>indicwiki</dbname>
		<base>https://tewiki.iiit.ac.in/index.php/%E0%B0%AE%E0%B1%8A%E0%B0%A6%E0%B0%9F%E0%B0%BF_%E0%B0%AA%E0%B1%87%E0%B0%9C%E0%B1%80</base>
		<generator>MediaWiki 1.34.0</generator>
		<case>first-letter</case>
		<namespaces>
			<namespace key="-2" case="first-letter">మీడియా</namespace>
			<namespace key="-1" case="first-letter">ప్రత్యేక</namespace>
			<namespace key="0" case="first-letter" />
			<namespace key="1" case="first-letter">చర్చ</namespace>
			<namespace key="2" case="first-letter">వాడుకరి</namespace>
			<namespace key="3" case="first-letter">వాడుకరి చర్చ</namespace>
			<namespace key="4" case="first-letter">Project</namespace>
			<namespace key="5" case="first-letter">Project చర్చ</namespace>
			<namespace key="6" case="first-letter">దస్త్రం</namespace>
			<namespace key="7" case="first-letter">దస్త్రంపై చర్చ</namespace>
			<namespace key="8" case="first-letter">మీడియావికీ</namespace>
			<namespace key="9" case="first-letter">మీడియావికీ చర్చ</namespace>
			<namespace key="10" case="first-letter">మూస</namespace>
			<namespace key="11" case="first-letter">మూస చర్చ</namespace>
			<namespace key="12" case="first-letter">సహాయం</namespace>
			<namespace key="13" case="first-letter">సహాయం చర్చ</namespace>
			<namespace key="14" case="first-letter">వర్గం</namespace>
			<namespace key="15" case="first-letter">వర్గం చర్చ</namespace>
			<namespace key="828" case="first-letter">మాడ్యూల్</namespace>
			<namespace key="829" case="first-letter">మాడ్యూల్ చర్చ</namespace>
			<namespace key="2300" case="first-letter">Gadget</namespace>
			<namespace key="2301" case="first-letter">Gadget talk</namespace>
			<namespace key="2302" case="case-sensitive">Gadget definition</namespace>
			<namespace key="2303" case="case-sensitive">Gadget definition talk</namespace>
			<namespace key="2600" case="first-letter">Topic</namespace>
		</namespaces>
	</siteinfo>\n'''

In [ ]:
page_id = 795001
user_id = 18884
username = 'SantoshJonnakuti'
def sha36(page_id):
	page_id = str(page_id).encode('utf-8')
	sha16 =sha1(page_id).hexdigest()
	sha10 =int(sha16, 16)

	chars =[]
	alphabets = string.digits +string.ascii_lowercase
	while sha10>0:
		sha10, r = divmod(sha10, 36)
		chars.append(alphabets[r])
	
	return ''.join(reversed(chars))


def writePage(title, wikiText, fobj):
	global user_id, username

	pglen = len(wikiText)
	time =datetime.now().strftime("%Y-%m-%dT%H-%M-%SZ")
	
	curPage ='''\n\n
	<page>
		<title>''' +title +'''</title>
		<ns>0</ns>
		<id>''' +str(page_id) +'''</id>
		<revision>
			<id>''' +str(page_id) +'''</id>
			<timestamp>'''+time+'''</timestamp>
			<contributor>
				<username>''' +username +'''</username>
				<id>''' +str(user_id) +'''</id>
			</contributor>
			<comment>xmlpage created</comment>
			<model>wikitext</model>
			<format>text/x-wiki</format>
			<text xml:space="preserve" bytes="''' +str(pglen) +'''">
			\n''' +wikiText +'''
			</text>
			<sha1>''' +sha36(page_id) +'''</sha1>
		</revision>
	</page>
	\n\n'''
	fobj.write(curPage)
	return

In [ ]:
fileLoader = FileSystemLoader('./')
env = Environment(loader=fileLoader)
env.globals.update(zip=zip)
template = env.get_template('template.j2')

In [ ]:
count = 0
wCount = 0
dCount = 0
iCount = 0
XMLFileObj = open('XML.xml', 'w')
XMLFileObj.write(tewiki+'\n')
XMLFileObj.close()
for ind in df.index[0:100]:
  print(ind)
  try:
    data = getData(df, ind)
    data = getProfessionalData(data)
    data = getMedalsData(data)
    text = template.render(data)
    title = data['name']
    text = text.strip()
    XMLFileObj = open('XML.xml', 'a')
    writePage(title, text, XMLFileObj)
    XMLFileObj.close()
    count += 1
    page_id += 1
  except Exception as e:
    print(e)
    if 'datetime' in str(e):
      dCount += 1
    elif 'list index' in str(e):
      iCount += 1
    else:
      wCount += 1

XMLFileObj = open('XML.xml', 'a')
XMLFileObj.write('</mediawiki>')
# XMLFileObj.close()
print(count, wCount, dCount, iCount)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
list index out of range
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
99 0 0 1


In [ ]:
data = getData(df, 1)
data = getProfessionalData(data)
data = getMedalsData(data)
text = template.render(data)
print(text.strip())
XMLFileObj = open('Usain_Bolt.xml', 'a')
XMLFileObj.write(tewiki+'\n')
title = data['name']
text = text.strip()
writePage(title, text, XMLFileObj)
XMLFileObj.write('</mediawiki>')
XMLFileObj.close()

{{Infobox sportsperson
| name =  రాగ్న్‌హిల్డ్ మార్గరెతే అమోడ్ట్
| full_name = రాగ్న్‌హిల్డ్ మార్గరెతే అమోడ్ట్
| image =  Ranghild_Aamodt_22.04.2009_small.jpg
| birth_date =  9/9/1980
| birth_place =  సర్ప్స్బోర్గ్
| country =  నార్వే
| sport =  హ్యాండ్‌బాల్
| height =  163.0
| weight =  nan| medaltemplates = 
{{MedalGold | 2008 బీజింగ్ | హ్యాండ్‌బాల్ ఉమెన్స్  హ్యాండ్‌బాల్}}
}}
}}
రాగ్న్‌హిల్డ్ మార్గరెతే అమోడ్ట్  [https://te.wikipedia.org/wiki/నార్వే నార్వే][NOR] దేశం కి చెందిన క్రీడాకారిణి.ఆమె [https://te.wikipedia.org/wiki/ఒలింపిక్_క్రీడలు ఒలింపిక్స్] లో నార్వే[NOR] తరపున హ్యాండ్‌బాల్ లో పాల్గొంది.
రాగ్న్‌హిల్డ్ మార్గరెతే అమోడ్ట్ 9/9/1980<ref>{{Cite web|url=https://dbpedia.org/page/Ragnhild_Aamodt|title= Dbpedia Page of Ragnhild Margrethe Aamodt|language=en}}</ref> తేదీన సర్ప్స్బోర్గ్ లో జన్మించింది.

=వ్యక్తిగత జీవితము=
ఈ క్రీడాకారిణి జాతీయత నార్వే.

=క్రీడా జీవితం=
రాగ్న్‌హిల్డ్ మార్గరెతే అమోడ్ట్ ఎత్తు 163.0 మీటర్లు.
రాగ్న్‌హిల్డ్ మార్గరెతే అమోడ్ట్ బరువు nan కిలోలు.
=== [https://te